In [1]:
import tensorflow as tf

# tf.saved_model -- Overview

**Classes**
- Asset: Represents a file asset to hermetically include in a SavedModel.
- LoadOptions: 加载`SavedModel`的选项
- SaveOptions: 保存`SavedModel`的选项

**Functions**

- contains_saved_model: 检查提供的导出目录是否包含`SavedModel`
- load: 从`export_dir`加载`SavedModel`
- save: 以`SavedModel`的格式导出可跟踪对象

**DATA**
- ASSETS_DIRECTORY = 'assets'
- ASSETS_KEY = 'saved_model_assets'
- CLASSIFY_INPUTS = 'inputs'
- CLASSIFY_METHOD_NAME = 'tensorflow/serving/classify'
- CLASSIFY_OUTPUT_CLASSES = 'classes'
- CLASSIFY_OUTPUT_SCORES = 'scores'
- DEBUG_DIRECTORY = 'debug'
- DEBUG_INFO_FILENAME_PB = 'saved_model_debug_info.pb'
- DEFAULT_SERVING_SIGNATURE_DEF_KEY = 'serving_default'
- GPU = 'gpu'
- PREDICT_INPUTS = 'inputs'
- PREDICT_METHOD_NAME = 'tensorflow/serving/predict'
- PREDICT_OUTPUTS = 'outputs'
- REGRESS_INPUTS = 'inputs'
- REGRESS_METHOD_NAME = 'tensorflow/serving/regress'
- REGRESS_OUTPUTS = 'outputs'
- SAVED_MODEL_FILENAME_PB = 'saved_model.pb'
- SAVED_MODEL_FILENAME_PBTXT = 'saved_model.pbtxt'
- SAVED_MODEL_SCHEMA_VERSION = 1
- SERVING = 'serve'
- TPU = 'tpu'
- TRAINING = 'train'
- VARIABLES_DIRECTORY = 'variables'
- VARIABLES_FILENAME = 'variables'

**FILE**： \tensorflow\_api\v2\saved_model\__init__.py

# tf.saved_model.save()
`tf.saved_model.save(obj, export_dir, signatures=None, options=None)`

**Docstring**

以[SavedModel格式](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md)导出可跟踪对象`obj`


Example usage:

```python
class Adder(tf.Module):

  @tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
  def add(self, x):
    return x + x + 1.

to_export = Adder()
tf.saved_model.save(to_export, '/tmp/adder')
```

The resulting SavedModel is then servable with an input named "x", its value
having any shape and dtype float32.

`signatures`控制`obj`中的哪些方法对那些使用`SavedModel`的程序是可用的，Python函数可以用`@tf.function(input_signature=…)`修饰并直接作为 signature 传递，也可以通过`get_concrete_function`调用被`@tf.function`修饰过的方法

If the `signatures` argument is omitted, `obj` will be searched for
`@tf.function`-decorated methods. If exactly one `@tf.function` is found, that
method will be used as the default signature for the SavedModel. This behavior
is expected to change in the future, when a corresponding
`tf.saved_model.load` symbol is added. At that point signatures will be
completely optional, and any `@tf.function` attached to `obj` or its
dependencies will be exported for use with `load`.

When invoking a signature in an exported SavedModel, `Tensor` arguments are
identified by name. These names will come from the Python function's argument
names by default. They may be overridden by specifying a `name=...` argument
in the corresponding `tf.TensorSpec` object. Explicit naming is required if
multiple `Tensor`s are passed through a single argument to the Python
function.

The outputs of functions used as `signatures` must either be flat lists, in
which case outputs will be numbered, or a dictionary mapping string keys to
`Tensor`, in which case the keys will be used to name outputs.

Signatures are available in objects returned by `tf.saved_model.load` as a
`.signatures` attribute. This is a reserved attribute: `tf.saved_model.save`
on an object with a custom `.signatures` attribute will raise an exception.

Since `tf.keras.Model` objects are also Trackable, this function can be
used to export Keras models. For example, exporting with a signature
specified:

```python
class Model(tf.keras.Model):

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
  def serve(self, serialized):
    ...

m = Model()
tf.saved_model.save(m, '/tmp/saved_model/')
```

Exporting from a function without a fixed signature:

```python
class Model(tf.keras.Model):

  @tf.function
  def call(self, x):
    ...

m = Model()
tf.saved_model.save(
    m, '/tmp/saved_model/',
    signatures=m.call.get_concrete_function(
        tf.TensorSpec(shape=[None, 3], dtype=tf.float32, name="inp")))
```

`tf.keras.Model` instances constructed from inputs and outputs already have a
signature and so do not require a `@tf.function` decorator or a `signatures`
argument. If neither are specified, the model's forward pass is exported.

```python
x = input_layer.Input((4,), name="x")
y = core.Dense(5, name="out")(x)
model = training.Model(x, y)
tf.saved_model.save(model, '/tmp/saved_model/')
# The exported SavedModel takes "x" with shape [None, 4] and returns "out"
# with shape [None, 5]
```

Variables must be tracked by assigning them to an attribute of a tracked
object or to an attribute of `obj` directly. TensorFlow objects (e.g. layers
from `tf.keras.layers`, optimizers from `tf.train`) track their variables
automatically. This is the same tracking scheme that `tf.train.Checkpoint`
uses, and an exported `Checkpoint` object may be restored as a training
checkpoint by pointing `tf.train.Checkpoint.restore` to the SavedModel's
"variables/" subdirectory. Currently, variables are the only stateful objects
supported by `tf.saved_model.save`, but others (e.g. tables) will be supported
in the future.

`tf.function` does not hard-code device annotations from outside the function
body, instead of using the calling context's device. This means for example
that exporting a model that runs on a GPU and serving it on a CPU will
generally work, with some exceptions. `tf.device` annotations inside the body
of the function will be hard-coded in the exported model; this type of
annotation is discouraged. Device-specific operations, e.g. with "cuDNN" in
the name or with device-specific layouts, may cause issues. Currently a
`DistributionStrategy` is another exception: active distribution strategies
will cause device placements to be hard-coded in a function. Exporting a
single-device computation and importing under a `DistributionStrategy` is
not currently supported, but may be in the future.

SavedModels exported with `tf.saved_model.save` [strip default-valued
attributes](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md#stripping-default-valued-attributes)
automatically, which removes one source of incompatibilities when the consumer
of a SavedModel is running an older TensorFlow version than the
producer. There are however other sources of incompatibilities which are not
handled automatically, such as when the exported model contains operations
which the consumer does not have definitions for.

A single tf.function can generate many ConcreteFunctions. If a downstream tool
wants to refer to all concrete functions generated by a single tf.function you
can use the `function_aliases` argument to store a map from the alias name to
all concrete function names.
E.g.
```python
class MyModel:
@tf.function
def func():
  ...

@tf.function
def serve():
  ...
  func()

model = MyModel()
signatures = {
    'serving_default': model.serve.get_concrete_function(),
}
options = tf.saved_model.SaveOptions(function_aliases={
    'my_func': func,
})
tf.saved_model.save(model, export_dir, signatures, options)
```

Args:
  obj: A trackable object to export.
  export_dir: A directory in which to write the SavedModel.
  signatures: Optional, either a `tf.function` with an input signature
    specified or the result of `f.get_concrete_function` on a
    `@tf.function`-decorated function `f`, in which case `f` will be used to
    generate a signature for the SavedModel under the default serving
    signature key. `signatures` may also be a dictionary, in which case it
    maps from signature keys to either `tf.function` instances with input
    signatures or concrete functions. The keys of such a dictionary may be
    arbitrary strings, but will typically be from the
    `tf.saved_model.signature_constants` module.
  options: Optional, `tf.saved_model.SaveOptions` object that specifies
    options for saving.

Raises:
  ValueError: If `obj` is not trackable.

@compatibility(eager)
Not well supported when graph building. From TensorFlow 1.x,
`tf.compat.v1.enable_eager_execution()` should run first. Calling
tf.saved_model.save in a loop when graph building from TensorFlow 1.x will
add new save operations to the default graph each iteration.

May not be called from within a function body.
@end_compatibility
File:      d:\programfiles\miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\saved_model\save.py
Type:      function

# tf.saved_model.load()
`tf.saved_model.load(export_dir, tags=None)`

**Docstring**

从`export_dir`加载一个具有`signatures`属性的可追踪的`SavedModel`，该`signatures`属性可以将 signature 键映射至函数；若`SavedModel`是通过`tf.saved_model.load`导出的，它也会指向它自己已经保存的可跟踪对象、函数、调试信息 (debug info) 等

对于从 Tensorflow 1.x 加载`SavedModels`，`tf.estimator.Estimator`或 Tensorflow 1.x 的 `SavedModel` API 的 `SavedModel` 有一个平面图，而非`tf.function`对象，这些`Savedmodel`利用`.signatures`和`.prune(feeds, fetches)`属性加载


**Consuming SavedModels asynchronously**

When consuming SavedModels asynchronously (the producer is a separate
process), the SavedModel directory will appear before all files have been
written, and `tf.saved_model.load` will fail if pointed at an incomplete
SavedModel. Rather than checking for the directory, check for
"saved_model_dir/saved_model.pb". This file is written atomically as the last
`tf.saved_model.save` file operation.

**Args**:
- export_dir: 要加载的`SavedModel`路径
- tags: A tag or sequence of tags identifying the MetaGraph to load. Optional if the SavedModel contains a single MetaGraph, as for those exported from `tf.saved_model.save`.

**File**:     \tensorflow\python\saved_model\load.py

**Type**:      function

### Example

**Loading Keras models**

Keras 模型是可追踪的，进而可以被保存至`SavedModel`；然而由`tf.saved_model.load`返回的并非 Keras 对象，例如不具有`.fit`、`.predict`等方法，然而某些属性和函数还是存在的，如`.variables`、`.trainable_variables`、`.__call__`

In [ ]:
from models.example import Baseline, model, outputs
path = "./test/tf.saved_model_test/from_models.example"
tf.saved_model.save(model, path)

In [ ]:
imported_model = tf.saved_model.load(path)
inputs = tf.constant(range(1, 25), shape=[2, 3, 4, 1], dtype=tf.float32)
outputs_ = imported_model(inputs)
print(outputs)
print(outputs_)

与`SavedModel`相关联的 signature 可以作为函数使用：

In [ ]:
f = imported_model.signatures["serving_default"]
print(f(inputs))

使用`tf.saved_model.save`导出的对象具有可跟踪的对象和函数的属性

In [40]:
path = "./test/tf.saved_model_test/tmp1"
exported = tf.train.Checkpoint(v=tf.Variable(3.))
exported.f = tf.function(
    lambda x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)]
)
tf.saved_model.save(exported, path)
imported = tf.saved_model.load(path)
assert 3. == imported.v.numpy()
assert 6. == imported.f(x=tf.constant(2.)).numpy()

INFO:tensorflow:Assets written to: ./test/tf.saved_model_test/tmp1\assets
